In [9]:
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import normalize
import spacy
from sklearn.feature_extraction import text 

from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

import string
from gensim.utils import lemmatize

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import pattern3
import spacy

pd.options.display.max_colwidth = 500

In [10]:
data = pickle.load(open('eth_comments_large.pkl', 'rb'))

In [11]:
# Make a list of lists containing comment and the subreddit its from
# comments = [comment.body for forest in data['comments'] for comment in forest.list()] 

comments_list = []

for i, forest in enumerate(data['comments']):
    subreddit = str(data['subreddit'][i])
    for comment in forest.list():
        comments_list.append([subreddit,comment.body])

comments_all = [comment[1] for comment in comments_list]

#the_ = [comment[1] for comment in comments_all if comment[0] in ['the_mueller','the_donald']]

In [12]:
comments = [comment for comment in comments_all] # if len(comment) > 300] 

In [13]:
comments = [s.translate(str.maketrans('', '', string.punctuation)) for s in comments]

In [14]:
comments = [comment.replace('\n',' ').replace('*','').replace("\'","").replace('"','').replace('>','').replace('\\',' ') for comment in comments]

In [15]:
additional_stop_words = ['deleted','don','https','pastebin','com','reddit','http','www','ethereum','eth','like','pron']
stop_words = text.ENGLISH_STOP_WORDS.union(additional_stop_words)
comments = [" ".join([word for word in comment.split(" ") if word not in stop_words]) for comment in comments]

In [16]:
nlp = spacy.load('en', disable=['parser', 'ner'])

comments = [" ".join([word.lemma_ for word in nlp(comment)]) for comment in comments]
#comments = [" ".join([stemmer.stem(word) for word in comment.split(" ")]) for comment in comments]
#comments = [[stemmer.stem(word) for word in comment.split(" ")] for comment in comments]

In [37]:
len(comments)

79139

In [48]:
# Create a CountVectorizer for parsing/counting words

count_vectorizer = CountVectorizer(ngram_range=(1, 1), 
                                   stop_words=stop_words, token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer.fit(comments)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'show', 'an', 'noone', 'between', 'around', 'former', 'beforehand', 'last', 'thereby', 'off', 'there', 'don', 'never', 'all', 'fifteen', 'name', 'formerly', 'your', 'hundred', 'the', 'couldnt', 'now', 'yourself', 'fifty', 'two', 'how', 'ever', 'un', 'some', 'four', 'among', 'fi...l', 'are', 'either', 'those', 'than', 'other', 'who', 'should', 'twenty', 'com', 'perhaps', 'they'}),
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [165]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words, token_pattern="\\b[a-z][a-z]+\\b")
tfidf.fit(comments)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=frozenset({'eleven', 'moreover', 'though', 'former', 'whole', 'nobody', 'side', 'indeed', 'found', 'whenever', 'couldnt', 'bottom', 'describe', 'so', 'by', 'anyone', 'latter', 'never', 'become', 'somehow', 'up', 'back', 'towards', 'only', 'thus', 'himself', 'throughout', 'last', 'go', 'to...'five', 'after', 'again', 'cry', 'no', 'www', 'thick', 'all', 'your', 'few', 'see', 'next', 'also'}),
        strip_accents=None, sublinear_tf=False,
        token_pattern='\\b[a-z][a-z]+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [49]:
doc_word = count_vectorizer.transform(comments).transpose()

In [50]:
doc_word.shape

(38548, 79139)

In [51]:
lsa = TruncatedSVD(3)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.04290569, 0.01052635, 0.0090341 ])

In [52]:
pd.DataFrame(doc_topic).shape

(38548, 3)

In [53]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
display_topics(lsa, count_vectorizer.get_feature_names(), 10)


Topic  0


IndexError: list index out of range

In [54]:
pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).head(5)

,0,1,2,3,4,5,6,7,8,9,...,79129,79130,79131,79132,79133,79134,79135,79136,79137,79138
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaaaaaaaaaaaaaaaaand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaaaaaaaanndd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaaand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
corpus = matutils.Sparse2Corpus(doc_word)

In [56]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [57]:
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=5)

In [58]:
lda.print_topics()

[(0,
  '0.022*"buy" + 0.022*"just" + 0.016*"people" + 0.015*"think" + 0.012*"money" + 0.012*"time" + 0.012*"crypto" + 0.011*"good" + 0.011*"know" + 0.011*"sell"'),
 (1,
  '0.023*"use" + 0.011*"value" + 0.009*"token" + 0.008*"just" + 0.008*"blockchain" + 0.007*"exchange" + 0.007*"transaction" + 0.007*"bitcoin" + 0.007*"need" + 0.006*"network"'),
 (2,
  '0.019*"bitcoin" + 0.006*"time" + 0.006*"gnt" + 0.006*"company" + 0.005*"community" + 0.005*"bullish" + 0.005*"ready" + 0.005*"new" + 0.005*"team" + 0.005*"think"'),
 (3,
  '0.036*"price" + 0.033*"btc" + 0.016*"thank" + 0.015*"market" + 0.013*"bitcoin" + 0.011*"thread" + 0.010*"post" + 0.010*"gdax" + 0.010*"comment" + 0.009*"cap"'),
 (4,
  '0.023*"fuck" + 0.017*"lol" + 0.013*"shit" + 0.009*"guy" + 0.008*"holy" + 0.006*"upvote" + 0.006*"check" + 0.005*"iota" + 0.005*"fucking" + 0.005*"lambo"')]

In [59]:
nmf_model = NMF(2)
doc_topic = nmf_model.fit_transform(doc_word)

In [60]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2"],
             columns = count_vectorizer.get_feature_names())
topic_word

ValueError: Shape of passed values is (2, 79139), indices imply (2, 38548)

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [61]:
display_topics(nmf_model, count_vectorizer.get_feature_names(), 20)


Topic  0


IndexError: list index out of range

In [ ]:
google_vec_file = '/Users/danielwang/metis/metisgh/NLP_embeddings/GoogleNews-vectors-negative300.bin'